<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/08_modern_CNNs_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convolutional Neural Networks

* **Convolution**: Given the input size ($n_h\times n_w$), and the kernel size ($k_h \times k_w$), the output size is as follows:
$$
(n_h - k_h + 1) \times (n_w - k_w + 1)
$$

